In [1]:
from hepmc import *
import numpy as np

In [2]:
sarge = densities.Sarge(2, 3, 100.)

In [3]:
eeqqg = densities.ee_qq_ng(1, 100., 5., .3)

In [4]:
rambo_mapping = phase_space.RamboOnDiet(100., 3)
mapped = phase_space.MappedDensity(eeqqg, rambo_mapping)

In [6]:
%time sarge_sample = ImportanceMC(sarge)(eeqqg.pdf, 10000)

Event 1	(avg. trials per event: 17.000000)
Event 2	(avg. trials per event: 10.500000)	XS = 75.441630 pb +- ( 24.087213 pb = 31.928278 %)
Event 3	(avg. trials per event: 7.333333)	XS = 80.513083 pb +- ( 33.956511 pb = 42.175147 %)
Event 4	(avg. trials per event: 6.250000)	XS = 79.451760 pb +- ( 31.925110 pb = 40.181753 %)
Event 5	(avg. trials per event: 5.200000)	XS = 104.411172 pb +- ( 31.031021 pb = 29.720020 %)
Event 6	(avg. trials per event: 5.500000)	XS = 82.464559 pb +- ( 28.947167 pb = 35.102555 %)
Event 7	(avg. trials per event: 4.857143)	XS = 93.793877 pb +- ( 27.705919 pb = 29.539155 %)
Event 8	(avg. trials per event: 4.375000)	XS = 114.599612 pb +- ( 28.621722 pb = 24.975409 %)
Event 9	(avg. trials per event: 4.222222)	XS = 105.971615 pb +- ( 29.108331 pb = 27.468045 %)
Event 10	(avg. trials per event: 3.900000)	XS = 103.369239 pb +- ( 30.388670 pb = 29.398175 %)
Event 20	(avg. trials per event: 4.200000)	XS = 69.266250 pb +- ( 16.279061 pb = 23.502155 %)
Event 30	(avg. trial

In [ ]:
densities.export_hepmc(100., sarge_sample, "../samples/qcd/2-3/sarge_weighted.hepmc")

In [ ]:
bound = sarge_sample.weights.max()

In [ ]:
%%time
# acceptance-rejection sampler
sarge_sample_unweighted = np.empty((10000, 12))
for i in range(10000):
    aprob = 0
    while np.random.random() > aprob:
        proposal = sarge.proposal()
        f = eeqqg.pdf(proposal)
        g = sarge.proposal_pdf(None, proposal)
        aprob = f/(bound*g)
        if aprob > 1:
            print("bound too low")
    sarge_sample_unweighted[i] = proposal
sarge_sample_unweighted = Sample(data=sarge_sample_unweighted, weights=np.full(10000, 1./10000))

In [ ]:
densities.export_hepmc(100., sarge_sample_unweighted, "../samples/qcd/2-3/sarge_unweighted.hepmc")

In [ ]:
rambo_sample = PlainMC(5)(mapped, 10000)
rambo_sample.integral, rambo_sample.integral_err

In [ ]:
ys = mapped.pdf(rambo_sample.data)
max_y = ys.max()

In [ ]:
rambo = AcceptRejectSampler(mapped, max_y, 5)
%time rambo_sample = rambo.sample(1000)

In [ ]:
rambo_sample.data = rambo_mapping.map(rambo_sample.data)
rambo_sample.weights = np.full(rambo_sample.size, 1./rambo_sample.size)

In [ ]:
densities.export_hepmc(100., rambo_sample, "../samples/qcd/2-3/rambo.hepmc")

In [ ]:
# just a local sampler (rambo)
met = DefaultMetropolis(5, mapped, proposals.Gaussian(5, .01))
%time met_sample = met.sample(100000, np.random.rand(5))

In [ ]:
met_sample.data = rambo_mapping.map(met_sample.data)
met_sample.weights = np.full(met_sample.size, 1./met_sample.size)
densities.export_hepmc(100., met_sample, "../samples/qcd/2-3/metropolis.hepmc")

In [ ]:
# now sarge + local in MC3
importance = DefaultMetropolis(3 * 4, eeqqg, proposal=sarge)  # using sarge, full momentum sample space
local = DefaultMetropolis(5, mapped, proposals.Gaussian(5, .01))  # using rambo, rambo sample space
update = MixingMarkovUpdate(5, [importance, local], target=mapped,
                            out_maps={0: rambo_mapping.map_inverse},
                            in_maps={0: rambo_mapping.map}) # rambo sample space, important to set target explicitly
%time mc3_sample = update.sample(100000, np.random.rand(5))

In [ ]:
mc3_sample.data = rambo_mapping.map(mc3_sample.data)
mc3_sample.weights = np.full(mc3_sample.size, 1./mc3_sample.size)
densities.export_hepmc(100., mc3_sample, "../samples/qcd/2-3/mc3.hepmc")